# Project 3 Data Analysis

In [ ]:
# Importing all the required imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

# Part 1 -- Particle Detection With a Homogeneous and Sampling Calorimeter

In [ ]:
# Importing the data files -- All energies are in MeV

electrons_100mev = pd.read_csv( "Electrons_100mev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

protons_100mev = pd.read_csv( "Protons_100mev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

neutrons_100mev = pd.read_csv( "Neutrons_100mev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

photons_100mev = pd.read_csv( "Photons_100mev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

electrons_10gev = pd.read_csv( "Electrons_10gev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

protons_10gev = pd.read_csv( "Protons_10gev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

neutrons_10gev = pd.read_csv( "Neutrons_10gev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

photons_10gev = pd.read_csv( "Photons_10gev.csv", comment="#",\
names=[ "True Energy", "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5" ] )

## Particle Classification

In [ ]:
# Here, we append the particle id onto each column
# 0 = electrons
# 1 = protons
# 2 = neutrons
# 3 = photons

# Define the particle id numbers
e = np.zeros(len(electrons_100mev),dtype=int)
p = np.ones(len(protons_100mev),dtype=int)
n = 2*np.ones(len(neutrons_100mev),dtype=int)
g = 3*np.ones(len(photons_100mev),dtype=int)

# Append it to the corresponding dataframe
electrons_100mev["Particle ID"] = e.tolist()
# Re-order columns to make it look better
electrons_100mev = electrons_100mev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

protons_100mev["Particle ID"] = p.tolist()
protons_100mev = protons_100mev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

neutrons_100mev["Particle ID"] = n.tolist()
neutrons_100mev = neutrons_100mev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

photons_100mev["Particle ID"] = g.tolist()
photons_100mev = photons_100mev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

electrons_10gev["Particle ID"] = e.tolist()
electrons_10gev = electrons_10gev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

protons_10gev["Particle ID"] = p.tolist()
protons_10gev = protons_10gev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

neutrons_10gev["Particle ID"] = n.tolist()
neutrons_10gev = neutrons_10gev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

photons_10gev["Particle ID"] = g.tolist()
photons_10gev = photons_10gev[["Particle ID","True Energy",\
                   "Homogeneous", "Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

In [ ]:
# Append all the dataframes together by energy and shuffle to create our dataset

# Appending
particles_100mev = pd.concat([electrons_100mev,protons_100mev,neutrons_100mev,photons_100mev],ignore_index=True)
particles_10gev = pd.concat([electrons_10gev,protons_10gev,neutrons_10gev,photons_10gev],ignore_index=True)

#Shuffling
particles_100mev = particles_100mev.sample(frac=1).reset_index(drop=True)
particles_10gev = particles_10gev.sample(frac=1).reset_index(drop=True)

In [ ]:
# Check to see if our data frame looks good
particles_10gev
# It looks great!

In [ ]:
# Create our testing and training datasets

#100 mev particles
y_100 = particles_100mev[["Particle ID"]] # Target
x_100 = particles_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]] # Parameters

#10 gev particles
y_10 = particles_10gev[["Particle ID"]]
x_10 = particles_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]]

x100m_train, x100m_test, y100m_train, y100m_test = train_test_split(x_100,y_100, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

x10g_train, x10g_test, y10g_train, y10g_test = train_test_split(x_10,y_10, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

In [ ]:
y100m_train # The target dataset looks good!

In [ ]:
# Define the neural network

def model(shape):
    # create model
    model = Sequential()
    # Add input with shape equal to 6 (number of energy counters)
    model.add(Input(shape=(shape,)))
    # Dense layer with 100 inputs and relu activation function
    model.add(Dense(100,kernel_initializer = 'normal',activation ='relu'))
    # Dropout layer to reduce overtraining
    model.add(Dropout(0.1)) 
    # Dense layer with 100 inputs and relu activation function
    model.add(Dense(100,kernel_initializer = 'normal',activation ='relu'))
    # Dropout layer to reduce overtraining
    model.add(Dropout(0.1))
    # output layer with 4 inputs and softmax activation function for categorisation
    model.add(Dense(4,kernel_initializer ='normal',activation='softmax'))
    # Compile model with SCC loss function as this is a classification NN
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
    return model

model_100m = model(np.shape(x100m_train)[1]) # Activate 100 MeV model
model_10g = model(np.shape(x10g_train)[1]) # Activate 10 GeV model

model_100m.summary() # The model looks acceptable and has over 11,000 trainable parameters

In [ ]:
# Fit the 100 MeV model with 512 batch size, 20% validation split and 100 epochs
history_100m = model_100m.fit(x=x100m_train,y=y100m_train,batch_size=512,\
                                              epochs=120,validation_split=0.2,shuffle=True)

# Fit the 10 GeV model with 128 batch size, 20% validation split and 100 epochs
history_10g = model_10g.fit(x=x10g_train,y=y10g_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

In [ ]:
#Plotting the loss curves

fig,ax = plt.subplots(2,2,figsize=(12,9))
ax[0,0].plot(history_100m.history['loss'],label='Loss')
ax[0,0].plot(history_100m.history['val_loss'],label='Validation Loss')
ax[0,0].set_title("Loss Change over Time (100 MeV)")
ax[0,0].set_xlabel("Epoch")
ax[0,0].set_ylabel("Loss")
ax[0,0].legend()

ax[1,0].plot(history_100m.history['accuracy'],label='Accuracy')
ax[1,0].plot(history_100m.history['val_accuracy'],label='Validation Accuracy')
ax[1,0].set_title("Accuracy Change over Time (100 MeV)")
ax[1,0].set_xlabel("Epoch")
ax[1,0].set_ylabel("Accuracy")
ax[1,0].legend()

ax[0,1].plot(history_10g.history['loss'],label='Loss')
ax[0,1].plot(history_10g.history['val_loss'],label='Validation Loss')
ax[0,1].set_title("Loss Change over Time (10 GeV)")
ax[0,1].set_xlabel("Epoch")
ax[0,1].set_ylabel("Loss")
ax[0,1].legend()

ax[1,1].plot(history_10g.history['accuracy'],label='Accuracy')
ax[1,1].plot(history_10g.history['val_accuracy'],label='Validation Accuracy')
ax[1,1].set_title("Accuracy Change over Time (10 GeV)")
ax[1,1].set_xlabel("Epoch")
ax[1,1].set_ylabel("Accuracy")
ax[1,1].legend()

plt.tight_layout()
plt.show()

These loss curves are very satisfactory as they decrease to values of around 0.6-0.7 and show no signs of overtraining (the validation loss is similar to the loss). The accuracy also tends to around 70% for both energies too. The higher energy one, however, is slightly more stable than the lower energy one as it fluctuates less. It used to fluctuate more in earlier runs when the same batch size was used (128) but with such a large dataset that differs in detected energy a lot (c.f. energy resolution calculations below), a small batch size would cause large fluctuations as the various data would have huge influences rather than being cancelled out via the safety of large numbers. The large batch size seems to work except for the 2 outrageous peaks* in the validation accuracy although, considering these only appear in the validation and not in the training set, it is probably due to the fact that only 20% of the data is used in the validation set so it is likely that due to the random nature of it, a disproportionate amount of the the difficult particles (e.g. photons) have been included. 

Furthermore, there is no obvious difference between the distinction of particles for the high and low energy particles which is mildly surprising. It was expected that lower energy particles would be easier to distinguish as they are always absorbed in the detector whereas the higher energy ones often pass straight through. Presumably these effects cancelled each other out, as the neural network used this phenomenon as a marker to differentiate the highly penetrating particles from those that are easily absorbed. 

\*after running the code again, there may be no peaks or more of them -- it changes every run.

In [ ]:
# See how well the model predicts the testing data set
prediction_100m = np.argmax(model_100m.predict(x100m_test),axis=1)
prediction_10g = np.argmax(model_10g.predict(x10g_test),axis=1)

In [ ]:
#Plot confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig,ax = plt.subplots(1,2,figsize=(12,5))
#Define the confusion matrix and normalise it so that each row/column sums to 1
cmat100m = confusion_matrix(y100m_test,prediction_100m,normalize='true')
cmat10g = confusion_matrix(y10g_test,prediction_10g,normalize='true')
#dispay the matrix and redefine the display labels to show the particle type
cmatplot100m = ConfusionMatrixDisplay(confusion_matrix=cmat100m,display_labels=['electron','proton','neutron','photon'])
cmatplot10g = ConfusionMatrixDisplay(confusion_matrix=cmat10g,display_labels=['electron','proton','neutron','photon'])

cmatplot100m.plot(ax=ax[0])
ax[0].set_title("100 MeV")
cmatplot10g.plot(ax=ax[1])
ax[1].set_title("10 GeV")
plt.show()

From these confusion matrices, it is clear to see that each energy had its strengths and weaknesses. Starting with 100 MeV, it does a fantastic job at differentiating between the hadrons, neutrons and protons, but not very good with the elementary particles, electrons and photons. 100 MeV is almost 1/10 the mass of the hadrons whilst 200 times the mass of the electron and deep into the gamma region of the EM spectrum. As a result, most of the protons neatly deposit their energy in both the homogeneous & sampling calorimeters whilst the neutron deposits in only the sampling calorimeter and the electrons & photons don't deposit energy in the sampling calorimeter at all. The model detects this and is therefore able to make good distinction between the protons and neutrons. The electrons and photons, however, are harder to differentiate because at low energies, the photon penetration is similar to that of the electron and they both shower in the homogeneous calorimeter but just do not have enough energy to penetrate into the sampling calorimeter. Thus, their energy deposits are incredibly similar and the neural network has found it easier to assume most particles are electrons hence the 28% efficiency of detecting photons and 60% chance it confuses it for an electron. I suspect the model predicts photons when they penetrate the first layer of the sampling calorimeter, but when energy is deposited in the homogeneous one and not the sampling one, then it assumes it's an electron. Therefore, it detects more electrons.

At 10 GeV, it is worse at differentiating between neutrons and protons but can still differentiate the hadrons from the elementary particles. Likely 10 GeV has increased the penetrating power so much that many protons simply pass through the homogeneous calorimeter without depositing much energy, thus making them indistinguishable from neutrons. Moreover, many particles simply escape the detectors altogether due to their high energies so it could be that a lot of the confusion comes from both types of particle depositing little energy. Conversely, the electron-photon differentiation is superior at high energies, likely due to the fact that the photon penetrating power is so high that it manages to pass through the lead plates of the sampling calorimeter and deposit there. The electron still deposits too much energy in the homogeneous one to penetrate hence the neural network is better.

To improve particle identification, an additional layer that induces specific particle reactions could be used. 

## Energy Resolution

In [ ]:
# Calculating the energy resolution for each particle separately and then all of them together

# Getting the data required 
E_true_100 = electrons_100mev['True Energy'] # Same for every particle
E_true_10g = electrons_10gev['True Energy']

# Total detected energy
E_electron_det_100 = electrons_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_proton_det_100 = protons_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_neutron_det_100 = neutrons_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_photon_det_100 = photons_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_total_det_100 = pd.concat([E_electron_det_100,E_proton_det_100,E_neutron_det_100,E_photon_det_100],\
                           ignore_index=True)

E_electron_det_10g = electrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_proton_det_10g = protons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_neutron_det_10g = neutrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_photon_det_10g = photons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_total_det_10g = pd.concat([E_electron_det_10g,E_proton_det_10g,E_neutron_det_10g,E_photon_det_10g],\
                           ignore_index=True)

# Remove zero values to avoid divide by 0 errors
E_electron_det_100 = E_electron_det_100.drop(np.where(E_electron_det_100==0)[0])
E_proton_det_100 = E_proton_det_100.drop(np.where(E_proton_det_100==0)[0])
E_neutron_det_100 = E_neutron_det_100.drop(np.where(E_neutron_det_100==0)[0])
E_photon_det_100 = E_photon_det_100.drop(np.where(E_photon_det_100==0)[0])
E_total_det_100 = E_total_det_100.drop(np.where(E_total_det_100==0)[0])

E_electron_det_10g = E_electron_det_10g.drop(np.where(E_electron_det_10g==0)[0])
E_proton_det_10g = E_proton_det_10g.drop(np.where(E_proton_det_10g==0)[0])
E_neutron_det_10g = E_neutron_det_10g.drop(np.where(E_neutron_det_10g==0)[0])
E_photon_det_10g = E_photon_det_10g.drop(np.where(E_photon_det_10g==0)[0])
E_total_det_10g = E_total_det_10g.drop(np.where(E_total_det_10g==0)[0])

# Calibrate the energy
E_electron_cal_100 = np.mean(E_true_100[:len(E_electron_det_100)]/np.asarray(E_electron_det_100))*E_electron_det_100 
E_proton_cal_100 = np.mean(E_true_100[:len(E_proton_det_100)]/np.asarray(E_proton_det_100))*E_proton_det_100
E_neutron_cal_100 = np.mean(E_true_100[:len(E_neutron_det_100)]/np.asarray(E_neutron_det_100))*E_neutron_det_100
E_photon_cal_100 = np.mean(E_true_100[:len(E_photon_det_100)]/np.asarray(E_photon_det_100))*E_photon_det_100
E_total_cal_100 = np.mean(np.asarray(4*list(E_true_100))[:len(E_total_det_100)]/np.asarray(E_total_det_100))*E_total_det_100

E_electron_cal_10g = np.mean(E_true_10g[:len(E_electron_det_10g)]/np.asarray(E_electron_det_10g))*E_electron_det_10g 
E_proton_cal_10g = np.mean(E_true_10g[:len(E_proton_det_10g)]/np.asarray(E_proton_det_10g))*E_proton_det_10g
E_neutron_cal_10g = np.mean(E_true_10g[:len(E_neutron_det_10g)]/np.asarray(E_neutron_det_10g))*E_neutron_det_10g
E_photon_cal_10g = np.mean(E_true_10g[:len(E_photon_det_10g)]/np.asarray(E_photon_det_10g))*E_photon_det_10g
E_total_cal_10g = np.mean(np.asarray(4*list(E_true_10g))[:len(E_total_det_10g)]/np.asarray(E_total_det_10g))*E_total_det_10g

In [ ]:
# Calculating the weighted differences of the calibrated and true energy
electron_diff_100 = (np.asarray(E_electron_cal_100)-E_true_100[:len(E_electron_det_100)])/E_true_100[:len(E_electron_det_100)]
proton_diff_100 = (np.asarray(E_proton_cal_100)-E_true_100[:len(E_proton_det_100)])/E_true_100[:len(E_proton_det_100)]
neutron_diff_100 = (np.asarray(E_neutron_cal_100)-E_true_100[:len(E_neutron_det_100)])/E_true_100[:len(E_neutron_det_100)]
photon_diff_100 = (np.asarray(E_photon_cal_100)-E_true_100[:len(E_photon_det_100)])/E_true_100[:len(E_photon_det_100)]
total_diff_100 = (np.asarray(E_total_cal_100)-np.asarray(4*list(E_true_100))[:len(E_total_det_100)])\
                    /np.asarray(4*list(E_true_100))[:len(E_total_det_100)]

electron_diff_10g = (np.asarray(E_electron_cal_10g)-E_true_10g[:len(E_electron_det_10g)])/E_true_10g[:len(E_electron_det_10g)]
proton_diff_10g = (np.asarray(E_proton_cal_10g)-E_true_10g[:len(E_proton_det_10g)])/E_true_10g[:len(E_proton_det_10g)]
neutron_diff_10g = (np.asarray(E_neutron_cal_10g)-E_true_10g[:len(E_neutron_det_10g)])/E_true_10g[:len(E_neutron_det_10g)]
photon_diff_10g = (np.asarray(E_photon_cal_10g)-E_true_10g[:len(E_photon_det_10g)])/E_true_10g[:len(E_photon_det_10g)]
total_diff_10g = (np.asarray(E_total_cal_10g)-np.asarray(4*list(E_true_10g))[:len(E_total_det_10g)])\
                    /np.asarray(4*list(E_true_10g))[:len(E_total_det_10g)]

# Calculating the energy resolution
electron_res_100 = np.std(electron_diff_100)
proton_res_100 = np.std(proton_diff_100)
neutron_res_100 = np.std(neutron_diff_100)
photon_res_100 = np.std(photon_diff_100)
total_res_100 = np.std(total_diff_100)

electron_res_10g = np.std(electron_diff_10g)
proton_res_10g = np.std(proton_diff_10g)
neutron_res_10g = np.std(neutron_diff_10g)
photon_res_10g = np.std(photon_diff_10g)
total_res_10g = np.std(total_diff_10g)

#Plotting the Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

ax[0,0].hist(electron_diff_100,label='100 MeV Resolution = {:.4f}'.format(electron_res_100),bins=50)
ax[0,0].hist(electron_diff_10g,label='10 GeV Resolution = {:.4f}'.format(electron_res_10g),alpha=0.9,bins=50)
ax[0,0].set_title("Calibration Energy histogram of Electrons")
ax[0,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_ylabel("Frequency")
ax[0,0].legend()

ax[1,0].hist(proton_diff_100,label='100 MeV Resolution = {:.4f}'.format(proton_res_100),bins=10)
ax[1,0].hist(proton_diff_10g,label='10 GeV Resolution = {:.4f}'.format(proton_res_10g),alpha=0.9,bins=50)
ax[1,0].set_title("Calibration Energy histogram of Protons")
ax[1,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_ylabel("Frequency")
ax[1,0].legend()

ax[0,1].hist(neutron_diff_100,label='100 MeV Resolution = {:.4f}'.format(neutron_res_100),bins=50)
ax[0,1].hist(neutron_diff_10g,label='10 GeV Resolution = {:.4f}'.format(neutron_res_10g),alpha=0.8,bins=50)
ax[0,1].set_title("Calibration Energy histogram of Neutrons")
ax[0,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_ylabel("Frequency")
ax[0,1].set_xscale('log') # Logarithmic scale to see the resolution better
ax[0,1].legend()

ax[1,1].hist(photon_diff_100,label='100 MeV Resolution = {:.4f}'.format(photon_res_100),bins=50)
ax[1,1].hist(photon_diff_10g,label='10 GeV Resolution = {:.4f}'.format(photon_res_10g),alpha=0.8,bins=50)
ax[1,1].set_title("Calibration Energy histogram of Photons")
ax[1,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_ylabel("Frequency")
ax[1,1].legend()

plt.tight_layout()
plt.show()

plt.hist(total_diff_100,label='100 MeV Resolution = {:.4f}'.format(total_res_100),bins=50)
plt.hist(total_diff_10g,label='10 GeV Resolution = {:.4f}'.format(total_res_10g),alpha=0.8,bins=50)
plt.title("Calibration Energy Histogram of all Particles")
plt.xlabel("($E_{cal}-E_{true})/E_{true}$")
plt.ylabel("Frequency")
plt.xscale('log') 
plt.legend()
plt.show()

As can be seen in the graphs above, the energy resolution is terrible -- especially for uncharged particles which see their resolutions 2 orders of magnitude greater than the charged ones. The reason for this is because uncharged particles go straight through the material without interacting much and therefore do not deposit as much energy. This reduces the resolution dramatically. This becomes incredibly apparent at high energies as the orange graphs are shifted significantly to the right, so much for the photons & neutrons that the scale must be logarithmic to even provide more than a single line for the low energy. In more positive news, however, the 100 MeV protons and electrons have acceptable resolutions which are gaussian centred around 0. On top of this, the 10 GeV photons also have a very good resolution which seems surprising at first but, upon further thinking, it is potentially because the photons shower enough in the lead layer that the pair-produced particles have enough energy to deposit it into the sampling calorimeter thus less energy is missing. On top of this, fewer photons may be scattering away as pair production is the solely dominant effect at this energy range (rather than compton scattering).

# Extension Part

## Using a Graphite target

In this section, a graphite target is used to induce scattering/interaction events before entering the calorimeters to aid in the classification of particles. The motivation behind this is that greater range of particles produced in the interactions (or the lack of interactions) will give more clues for the neural network to guess which particle is responsible for the energy deposited. On top of this, energy loss via scattering within the target will also offer clues to the neural network on top of the regular data.

In [ ]:
# Importing the data files -- All energies are in MeV

# Get rid of column 0 which contains the true energy
electrons_100mev_Ctarget = pd.read_csv( "Electrons_100mev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

protons_100mev_Ctarget = pd.read_csv( "Protons_100mev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

neutrons_100mev_Ctarget = pd.read_csv( "Neutrons_100mev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

photons_100mev_Ctarget = pd.read_csv( "Photons_100mev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

electrons_10gev_Ctarget = pd.read_csv( "Electrons_10gev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

protons_10gev_Ctarget = pd.read_csv( "Protons_10gev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

neutrons_10gev_Ctarget = pd.read_csv( "Neutrons_10gev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

photons_10gev_Ctarget = pd.read_csv( "Photons_10gev_Ctarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT" ] )

In [ ]:
# Append the data alongside the non-target data.

graphite_electrons_100mev = pd.concat([electrons_100mev,electrons_100mev_Ctarget],axis=1)
graphite_protons_100mev = pd.concat([protons_100mev,protons_100mev_Ctarget],axis=1)
graphite_neutrons_100mev = pd.concat([neutrons_100mev,neutrons_100mev_Ctarget],axis=1)
graphite_photons_100mev = pd.concat([photons_100mev,photons_100mev_Ctarget],axis=1)
graphite_electrons_10gev = pd.concat([electrons_10gev,electrons_10gev_Ctarget],axis=1)
graphite_protons_10gev = pd.concat([protons_10gev,protons_10gev_Ctarget],axis=1)
graphite_neutrons_10gev = pd.concat([neutrons_10gev,neutrons_10gev_Ctarget],axis=1)
graphite_photons_10gev = pd.concat([photons_10gev,photons_10gev_Ctarget],axis=1)

In [ ]:
# Append all the dataframes together by energy and shuffle to create our dataset

# Appending
graphite_particles_100mev = pd.concat([graphite_electrons_100mev,graphite_protons_100mev,graphite_neutrons_100mev,\
                                  graphite_photons_100mev],ignore_index=True)
graphite_particles_10gev = pd.concat([graphite_electrons_10gev,graphite_protons_10gev,graphite_neutrons_10gev,\
                                 graphite_photons_10gev],ignore_index=True)

# Shuffling
graphite_particles_100mev = graphite_particles_100mev.sample(frac=1).reset_index(drop=True)
graphite_particles_10gev = graphite_particles_10gev.sample(frac=1).reset_index(drop=True)

In [ ]:
# Check to see if our data frame looks good
graphite_particles_10gev
# It looks great!

In [ ]:
# Create our testing and training datasets

Ct_y_100 = graphite_particles_100mev[["Particle ID"]] # Target
Ct_x_100 = graphite_particles_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                           "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]] # Parameters

Ct_y_10 = graphite_particles_10gev[["Particle ID"]]
Ct_x_10 = graphite_particles_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                         "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]]

Ct_x100m_train, Ct_x100m_test, Ct_y100m_train, Ct_y100m_test = train_test_split(Ct_x_100,Ct_y_100, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

Ct_x10g_train, Ct_x10g_test, Ct_y10g_train, Ct_y10g_test = train_test_split(Ct_x_10,Ct_y_10, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

In [ ]:
# Use the same model structure but now with more data!

model_Ct_100m = model(np.shape(Ct_x100m_train)[1]) # 100 mev
model_Ct_10g = model(np.shape(Ct_x10g_train)[1]) # 10 gev

model_Ct_100m.summary()
# This looks the same so all is well 

In [ ]:
# For 100 mev, use 100 epochs with 20% validation split and batch size 128 (this loss is less jagged)
history_Ct_100m = model_Ct_100m.fit(x=Ct_x100m_train,y=Ct_y100m_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

# For 10 gev, use the same parameters as above
history_Ct_10g = model_Ct_10g.fit(x=Ct_x10g_train,y=Ct_y10g_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

In [ ]:
#Plotting the loss curves

fig,ax = plt.subplots(2,2,figsize=(12,9))
ax[0,0].plot(history_Ct_100m.history['loss'],label='Loss')
ax[0,0].plot(history_Ct_100m.history['val_loss'],label='Validation Loss')
ax[0,0].set_title("Loss Change over Time (100 MeV)")
ax[0,0].set_xlabel("Epoch")
ax[0,0].set_ylabel("Loss")
ax[0,0].legend()

ax[1,0].plot(history_Ct_100m.history['accuracy'],label='Accuracy')
ax[1,0].plot(history_Ct_100m.history['val_accuracy'],label='Validation Accuracy')
ax[1,0].set_title("Accuracy Change over Time (100 MeV)")
ax[1,0].set_xlabel("Epoch")
ax[1,0].set_ylabel("Accuracy")
ax[1,0].legend()

ax[0,1].plot(history_Ct_10g.history['loss'],label='Loss')
ax[0,1].plot(history_Ct_10g.history['val_loss'],label='Validation Loss')
ax[0,1].set_title("Loss Change over Time (10 GeV)")
ax[0,1].set_xlabel("Epoch")
ax[0,1].set_ylabel("Loss")
ax[0,1].legend()

ax[1,1].plot(history_Ct_10g.history['accuracy'],label='Accuracy')
ax[1,1].plot(history_Ct_10g.history['val_accuracy'],label='Validation Accuracy')
ax[1,1].set_title("Accuracy Change over Time (10 GeV)")
ax[1,1].set_xlabel("Epoch")
ax[1,1].set_ylabel("Accuracy")
ax[1,1].legend()

plt.tight_layout()
plt.show()

From these loss curves, it is found that the accuracy is better than it was without the target for both the energies. The accuracy is between 86%-88% for 100 MeV (compared to 65%-70% without the target) and for 10 GeV, the accuracy lies between 77%-80%  (compared to 68%-70% without the target). This is a great success and it is stronger for 200 MeV than for 10 GeV. This is to be expected as the high energy particles will often pass straight through the target and so won't show much difference in the calorimeters whereas the 100 MeV particles interact more often thus producing differences. Like those above, the loss curves do not show any overtraining and also aren't decreasing any more meaning the number of epochs is sufficient.  

In [ ]:
# Using the model on the testing data
prediction_Ct_100m = np.argmax(model_Ct_100m.predict(Ct_x100m_test),axis=1)
prediction_Ct_10g = np.argmax(model_Ct_10g.predict(Ct_x10g_test),axis=1)

In [ ]:
#Plot confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig,ax = plt.subplots(1,2,figsize=(12,5))
#Define the confusion matrix and normalise it so that each row/column sums to 1
Ct_cmat100m = confusion_matrix(Ct_y100m_test,prediction_Ct_100m,normalize='true')
Ct_cmat10g = confusion_matrix(Ct_y10g_test,prediction_Ct_10g,normalize='true')
#dispay the matrix and redefine the display labels to show the particle type
Ct_cmatplot100m = ConfusionMatrixDisplay(confusion_matrix=Ct_cmat100m,display_labels=['electron','proton','neutron','photon'])
Ct_cmatplot10g = ConfusionMatrixDisplay(confusion_matrix=Ct_cmat10g,display_labels=['electron','proton','neutron','photon'])

Ct_cmatplot100m.plot(ax=ax[0])
ax[0].set_title("100 MeV")
Ct_cmatplot10g.plot(ax=ax[1])
ax[1].set_title("10 GeV")
plt.show()

From these matrices, it is clear to see that all particles saw similar benefits at both energy ranges. The protons are almost entirely distinguishable at 200 MeV and the neutrons also see a >90% efficiency. It is a modest effect compared to the original data but it is hard to improve on 91% so it is still an achievement. At 10 GeV, the hadrons have a lot higher success rate and still exhibit the same confusion effects meaning the main differentiation methods still come from how they interact in the sampling calorimeter. An interesting thing to note is that, previously, the protons and neutrons were very similar (74% & 72%, respectively) but now they are slightly further apart. This could just be a coincidence but it could also be to do with the fact that protons interact more with the target than the neutrons thus creating more EM particles to deposit energy in the homogeneous calorimeter.

The largest difference comes from the elementary particles as the photon has now above 60% accuracy at both energies whilst the electron remains comparable. This means that the neural network is no longer labelling every homogeneous calorimeter deposited energy as an electron but is now differentiating between the two. The difference is likely due to the fact that the electrons interact strongly with the graphite as graphite foil is, in fact, used as an electron diffraction grating in many experiments. Thus, particles that deposit a lot of energy is the homogeneous calorimeter are labelled as photons and particles that deposit little (due to scattering) are labelled as electrons. This doesn't appear to be energy dependent as the distinction quality is highly comparable for the two energies.

All in all, the addition of target data is a huge success as it improved particle detection.

## Energy Resolution

In [ ]:
# Calculating the energy resolution for each particle separately and then all of them together

# Getting the data required 
E_true_100 = electrons_100mev['True Energy'] # Same for every particle
# Total detected energy
E_gelectron_det_100 = graphite_electrons_100mev[["HomogeneousCT","Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]].sum(axis=1)
E_gproton_det_100 = graphite_protons_100mev[["HomogeneousCT","Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]].sum(axis=1)
E_gneutron_det_100 = graphite_neutrons_100mev[["HomogeneousCT","Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]].sum(axis=1)
E_gphoton_det_100 = graphite_photons_100mev[["HomogeneousCT","Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT"]].sum(axis=1)
E_gtotal_det_100 = pd.concat([E_gelectron_det_100,E_gproton_det_100,E_gneutron_det_100,E_gphoton_det_100],\
                           ignore_index=True)

E_gelectron_det_10g = graphite_electrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_gproton_det_10g = graphite_protons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_gneutron_det_10g = graphite_neutrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_gphoton_det_10g = graphite_photons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_gtotal_det_10g = pd.concat([E_gelectron_det_10g,E_gproton_det_10g,E_gneutron_det_10g,E_gphoton_det_10g],\
                           ignore_index=True)

# Remove zero values to avoid divide by 0 errors
E_gelectron_det_100 = E_gelectron_det_100.drop(np.where(E_gelectron_det_100==0)[0])
E_gproton_det_100 = E_gproton_det_100.drop(np.where(E_gproton_det_100==0)[0])
E_gneutron_det_100 = E_gneutron_det_100.drop(np.where(E_gneutron_det_100==0)[0])
E_gphoton_det_100 = E_gphoton_det_100.drop(np.where(E_gphoton_det_100==0)[0])
E_gtotal_det_100 = E_gtotal_det_100.drop(np.where(E_gtotal_det_100==0)[0])

E_gelectron_det_10g = E_gelectron_det_10g.drop(np.where(E_gelectron_det_10g==0)[0])
E_gproton_det_10g = E_gproton_det_10g.drop(np.where(E_gproton_det_10g==0)[0])
E_gneutron_det_10g = E_gneutron_det_10g.drop(np.where(E_gneutron_det_10g==0)[0])
E_gphoton_det_10g = E_gphoton_det_10g.drop(np.where(E_gphoton_det_10g==0)[0])
E_gtotal_det_10g = E_gtotal_det_10g.drop(np.where(E_gtotal_det_10g==0)[0])

# Calibrate the energy
E_gelectron_cal_100 = np.mean(E_true_100[:len(E_gelectron_det_100)]/np.asarray(E_gelectron_det_100))*E_gelectron_det_100 
E_gproton_cal_100 = np.mean(E_true_100[:len(E_gproton_det_100)]/np.asarray(E_gproton_det_100))*E_gproton_det_100
E_gneutron_cal_100 = np.mean(E_true_100[:len(E_gneutron_det_100)]/np.asarray(E_gneutron_det_100))*E_gneutron_det_100
E_gphoton_cal_100 = np.mean(E_true_100[:len(E_gphoton_det_100)]/np.asarray(E_gphoton_det_100))*E_gphoton_det_100
E_gtotal_cal_100 = np.mean(np.asarray(4*list(E_true_100))[:len(E_gtotal_det_100)]/np.asarray(E_gtotal_det_100))*E_gtotal_det_100

E_gelectron_cal_10g = np.mean(E_true_10g[:len(E_gelectron_det_10g)]/np.asarray(E_gelectron_det_10g))*E_gelectron_det_10g 
E_gproton_cal_10g = np.mean(E_true_10g[:len(E_gproton_det_10g)]/np.asarray(E_gproton_det_10g))*E_gproton_det_10g
E_gneutron_cal_10g = np.mean(E_true_10g[:len(E_gneutron_det_10g)]/np.asarray(E_gneutron_det_10g))*E_gneutron_det_10g
E_gphoton_cal_10g = np.mean(E_true_10g[:len(E_gphoton_det_10g)]/np.asarray(E_gphoton_det_10g))*E_gphoton_det_10g
E_gtotal_cal_10g = np.mean(np.asarray(4*list(E_true_10g))[:len(E_gtotal_det_10g)]/np.asarray(E_total_det_10g))*E_gtotal_det_10g

In [ ]:
# Calculating the weighted differences of the calibrated and true energy
electron_gdiff_100 = (np.asarray(E_gelectron_cal_100)-E_true_100[:len(E_gelectron_det_100)])/E_true_100[:len(E_gelectron_det_100)]
proton_gdiff_100 = (np.asarray(E_gproton_cal_100)-E_true_100[:len(E_gproton_det_100)])/E_true_100[:len(E_gproton_det_100)]
neutron_gdiff_100 = (np.asarray(E_gneutron_cal_100)-E_true_100[:len(E_gneutron_det_100)])/E_true_100[:len(E_gneutron_det_100)]
photon_gdiff_100 = (np.asarray(E_gphoton_cal_100)-E_true_100[:len(E_gphoton_det_100)])/E_true_100[:len(E_gphoton_det_100)]
total_gdiff_100 = (np.asarray(E_gtotal_cal_100)-np.asarray(4*list(E_true_100))[:len(E_gtotal_det_100)])\
                    /np.asarray(4*list(E_true_100))[:len(E_gtotal_det_100)]

electron_gdiff_10g = (np.asarray(E_gelectron_cal_10g)-E_true_10g[:len(E_gelectron_det_10g)])/E_true_10g[:len(E_gelectron_det_10g)]
proton_gdiff_10g = (np.asarray(E_gproton_cal_10g)-E_true_10g[:len(E_gproton_det_10g)])/E_true_10g[:len(E_gproton_det_10g)]
neutron_gdiff_10g = (np.asarray(E_gneutron_cal_10g)-E_true_10g[:len(E_gneutron_det_10g)])/E_true_10g[:len(E_gneutron_det_10g)]
photon_gdiff_10g = (np.asarray(E_gphoton_cal_10g)-E_true_10g[:len(E_gphoton_det_10g)])/E_true_10g[:len(E_gphoton_det_10g)]
total_gdiff_10g = (np.asarray(E_gtotal_cal_10g)-np.asarray(4*list(E_true_10g))[:len(E_gtotal_det_10g)])\
                    /np.asarray(4*list(E_true_10g))[:len(E_gtotal_det_10g)]

# Calculating the energy resolution
electron_gres_100 = np.std(electron_gdiff_100)
proton_gres_100 = np.std(proton_gdiff_100)
neutron_gres_100 = np.std(neutron_gdiff_100)
photon_gres_100 = np.std(photon_gdiff_100)
total_gres_100 = np.std(total_gdiff_100)

electron_gres_10g = np.std(electron_gdiff_10g)
proton_gres_10g = np.std(proton_gdiff_10g)
neutron_gres_10g = np.std(neutron_gdiff_10g)
photon_gres_10g = np.std(photon_gdiff_10g)
total_gres_10g = np.std(total_gdiff_10g)

#Plotting the Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

ax[0,0].hist(electron_gdiff_100,label='100 MeV Resolution = {:.4f}'.format(electron_gres_100))
ax[0,0].hist(electron_gdiff_10g,label='10 GeV Resolution = {:.4f}'.format(electron_gres_10g),alpha=0.8)
ax[0,0].set_title("Calibration Energy histogram of Electrons")
ax[0,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_ylabel("Frequency")
ax[0,0].legend()

ax[1,0].hist(proton_gdiff_100,label='100 MeV Resolution = {:.4f}'.format(proton_gres_100))
ax[1,0].hist(proton_gdiff_10g,label='10 GeV Resolution = {:.4f}'.format(proton_gres_10g),alpha=0.8)
ax[1,0].set_title("Calibration Energy histogram of Protons")
ax[1,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_ylabel("Frequency")
ax[1,0].legend()

ax[0,1].hist(neutron_gdiff_100,label='100 MeV Resolution = {:.4f}'.format(neutron_gres_100))
ax[0,1].hist(neutron_gdiff_10g,label='10 GeV Resolution = {:.4f}'.format(neutron_gres_10g),alpha=0.7)
ax[0,1].set_title("Calibration Energy histogram of Neutrons")
ax[0,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_ylabel("Frequency")
ax[0,1].set_xscale('log')
ax[0,1].legend()

ax[1,1].hist(photon_gdiff_100,label='100 MeV Resolution = {:.4f}'.format(photon_gres_100))
ax[1,1].hist(photon_gdiff_10g,label='10 GeV Resolution = {:.4f}'.format(photon_gres_10g))
ax[1,1].set_title("Calibration Energy histogram of Photons")
ax[1,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_ylabel("Frequency")
ax[1,1].legend()

plt.tight_layout()
plt.show()

plt.hist(total_gdiff_100,label='100 MeV Resolution = {:.4f}'.format(total_gres_100))
plt.hist(total_gdiff_10g,label='10 GeV Resolution = {:.4f}'.format(total_gres_10g),alpha=0.8)
plt.title("Calibration Energy histogram of All Particles")
plt.xlabel("($E_{cal}-E_{true})/E_{true}$")
plt.ylabel("Frequency")
plt.xscale('log')
plt.legend()
plt.show()

The energy resolution is not as bad as initially feared. At low energies, the overall resolution has jumped from 9 to 13 which is certainly due to the energy deposited in the target rather than the calorimeters and also due to scattering causing them to fly off and miss the detector altogether. In fact, the proton resolution is still higher at high energies than at low energies meaning the majority of target interactions did not reduce the energy a noticeable amount. This hints at the idea that the majority of interactions are elastic scattering and that the neural network compares between the non-target data and the target data to differentiate the particle. 

As before, neutrons are still the large contributor to the poor resolution as their neutrality and low cross sections mean they penetrate deeply without depositing much energy.

## Using Tungsten

In [ ]:
# Importing the data files -- All energies are in MeV

# Get rid of column 0 which contains the true energy
electrons_100mev_Wtarget = pd.read_csv( "Electrons_100mev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

protons_100mev_Wtarget = pd.read_csv( "Protons_100mev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

neutrons_100mev_Wtarget = pd.read_csv( "Neutrons_100mev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

photons_100mev_Wtarget = pd.read_csv( "Photons_100mev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

electrons_10gev_Wtarget = pd.read_csv( "Electrons_10gev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

protons_10gev_Wtarget = pd.read_csv( "Protons_10gev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

neutrons_10gev_Wtarget = pd.read_csv( "Neutrons_10gev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

photons_10gev_Wtarget = pd.read_csv( "Photons_10gev_Wtarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT" ] )

In [ ]:
# Append the data alongside the non-target data.

tungsten_electrons_100mev = pd.concat([electrons_100mev,electrons_100mev_Wtarget],axis=1)
tungsten_protons_100mev = pd.concat([protons_100mev,protons_100mev_Wtarget],axis=1)
tungsten_neutrons_100mev = pd.concat([neutrons_100mev,neutrons_100mev_Wtarget],axis=1)
tungsten_photons_100mev = pd.concat([photons_100mev,photons_100mev_Wtarget],axis=1)
tungsten_electrons_10gev = pd.concat([electrons_10gev,electrons_10gev_Wtarget],axis=1)
tungsten_protons_10gev = pd.concat([protons_10gev,protons_10gev_Wtarget],axis=1)
tungsten_neutrons_10gev = pd.concat([neutrons_10gev,neutrons_10gev_Wtarget],axis=1)
tungsten_photons_10gev = pd.concat([photons_10gev,photons_10gev_Wtarget],axis=1)

In [ ]:
# Append all the dataframes together by energy and shuffle to create our dataset

#Appending
tungsten_particles_100mev = pd.concat([tungsten_electrons_100mev,tungsten_protons_100mev,tungsten_neutrons_100mev,\
                                  tungsten_photons_100mev],ignore_index=True)
tungsten_particles_10gev = pd.concat([tungsten_electrons_10gev,tungsten_protons_10gev,tungsten_neutrons_10gev,\
                                 tungsten_photons_10gev],ignore_index=True)

# Shuffling
tungsten_particles_100mev = tungsten_particles_100mev.sample(frac=1).reset_index(drop=True)
tungsten_particles_10gev = tungsten_particles_10gev.sample(frac=1).reset_index(drop=True)

In [ ]:
# Check to see if our data frame looks good
tungsten_particles_100mev
# It looks great!

In [ ]:
# Create our testing and training datasets

Wt_y_100 = tungsten_particles_100mev[["Particle ID"]]
Wt_x_100 = tungsten_particles_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                           "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]]

Wt_y_10 = tungsten_particles_10gev[["Particle ID"]]
Wt_x_10 = tungsten_particles_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                         "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]]

Wt_x100m_train, Wt_x100m_test, Wt_y100m_train, Wt_y100m_test = train_test_split(Wt_x_100,Wt_y_100, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

Wt_x10g_train, Wt_x10g_test, Wt_y10g_train, Wt_y10g_test = train_test_split(Wt_x_10,Wt_y_10, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

In [ ]:
# Again, use the same model structure
model_Wt_100m = model(np.shape(Wt_x100m_train)[1])
model_Wt_10g = model(np.shape(Wt_x10g_train)[1])

model_Wt_100m.summary()
# This looks the same so all is well 

In [ ]:
# Use a large batch size of 512 to remove the fluctuations and 80 epochs to remove overtraining
history_Wt_100m = model_Wt_100m.fit(x=Wt_x100m_train,y=Wt_y100m_train,batch_size=512,\
                                              epochs=80,validation_split=0.2,shuffle=True)


# Use batch size of 128 and with 100 epochs
history_Wt_10g = model_Wt_10g.fit(x=Wt_x10g_train,y=Wt_y10g_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

In [ ]:
#Plotting the loss curves

fig,ax = plt.subplots(2,2,figsize=(12,9))
ax[0,0].plot(history_Wt_100m.history['loss'],label='Loss')
ax[0,0].plot(history_Wt_100m.history['val_loss'],label='Validation Loss')
ax[0,0].set_title("Loss Change over Time (100 MeV)")
ax[0,0].set_xlabel("Epoch")
ax[0,0].set_ylabel("Loss")
ax[0,0].legend()

ax[1,0].plot(history_Wt_100m.history['accuracy'],label='Accuracy')
ax[1,0].plot(history_Wt_100m.history['val_accuracy'],label='Validation Accuracy')
ax[1,0].set_title("Accuracy Change over Time (100 MeV)")
ax[1,0].set_xlabel("Epoch")
ax[1,0].set_ylabel("Accuracy")
ax[1,0].legend()

ax[0,1].plot(history_Wt_10g.history['loss'],label='Loss')
ax[0,1].plot(history_Wt_10g.history['val_loss'],label='Validation Loss')
ax[0,1].set_title("Loss Change over Time (10 GeV)")
ax[0,1].set_xlabel("Epoch")
ax[0,1].set_ylabel("Loss")
ax[0,1].legend()

ax[1,1].plot(history_Wt_10g.history['accuracy'],label='Accuracy')
ax[1,1].plot(history_Wt_10g.history['val_accuracy'],label='Validation Accuracy')
ax[1,1].set_title("Accuracy Change over Time (10 GeV)")
ax[1,1].set_xlabel("Epoch")
ax[1,1].set_ylabel("Accuracy")
ax[1,1].legend()

plt.tight_layout()
plt.show()

After last time's success, it is disappointing to see that these loss curves are worse than for graphite. These ones are more comparable to the original model except it does have a 5% higher accuracy at 10 GeV than for the original. The reason for this is probably because tungsten is a relatively heavy element (A=74) compared to carbon (A=12) meaning it is more absorbent. Therefore, it is possible that many of the particles were absorbed entirely into the target at low energies. However, at 10 GeV, it appears many did interact and still deposited energy into the detector so it does have a greater accuracy.

In [ ]:
# Testing the model on the testing data
prediction_Wt_100m = np.argmax(model_Wt_100m.predict(Wt_x100m_test),axis=1)
prediction_Wt_10g = np.argmax(model_Wt_10g.predict(Wt_x10g_test),axis=1)

In [ ]:
#Plot confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig,ax = plt.subplots(1,2,figsize=(12,5))
#Define the confusion matrix and normalise it so that each row/column sums to 1
Wt_cmat100m = confusion_matrix(Wt_y100m_test,prediction_Wt_100m,normalize='true')
Wt_cmat10g = confusion_matrix(Wt_y10g_test,prediction_Wt_10g,normalize='true')
#dispay the matrix and redefine the display labels to show the particle type
Wt_cmatplot100m = ConfusionMatrixDisplay(confusion_matrix=Wt_cmat100m,display_labels=['electron','proton','neutron','photon'])
Wt_cmatplot10g = ConfusionMatrixDisplay(confusion_matrix=Wt_cmat10g,display_labels=['electron','proton','neutron','photon'])

Wt_cmatplot100m.plot(ax=ax[0])
ax[0].set_title("100 MeV")
Wt_cmatplot10g.plot(ax=ax[1])
ax[1].set_title("10 GeV")
plt.show()

As expected from the loss curves, the confusion matrices are similar to the original ones. At low energies, the tungsten does not induce enough showering scattering reactions to provide any different data to the original. In fact, looking at the data, it is seen that a lot of the values are 0 for the energy -- for all particles. Thus, it appears that at 100 MeV, no additional useful data is being added to the neural network thus it is giving a very similar performance as the original one. The tungsten block is absorbing too many particles.

The 10 GeV matrix is slightly more positive as it is better at differentiating between neutrons and even photons. Presumably, at these high energies, some of the uncharged neutrons and photons pass straight through the tungsten shield and are able to deposit their energies into the calorimeter. Therefore, any additional data will be interpreted as coming from either neutrons or photons -- the difference between those two being that the photons deposit their energy in the homogeneous calorimeter and the neutrons in the sampling calorimeter.

## Energy Calibration

In [ ]:
# Calculating the energy resolution for each particle separately and then all of them together

# Getting the data required 
E_true_100 = electrons_100mev['True Energy'] # Same for every particle
E_true_10g = electrons_10gev['True Energy']
# Total detected energy
E_welectron_det_100 = tungsten_electrons_100mev[["HomogeneousWT","Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]].sum(axis=1)
E_wproton_det_100 = tungsten_protons_100mev[["HomogeneousWT","Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]].sum(axis=1)
E_wneutron_det_100 = tungsten_neutrons_100mev[["HomogeneousWT","Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]].sum(axis=1)
E_wphoton_det_100 = tungsten_photons_100mev[["HomogeneousWT","Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT"]].sum(axis=1)
E_wtotal_det_100 = pd.concat([E_welectron_det_100,E_wproton_det_100,E_wneutron_det_100,E_wphoton_det_100],\
                           ignore_index=True)

E_welectron_det_10g = tungsten_electrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_wproton_det_10g = tungsten_protons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_wneutron_det_10g = tungsten_neutrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_wphoton_det_10g = tungsten_photons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_wtotal_det_10g = pd.concat([E_welectron_det_10g,E_wproton_det_10g,E_wneutron_det_10g,E_wphoton_det_10g],\
                           ignore_index=True)

# Remove zero values to avoid divide by 0 errors
E_welectron_det_100 = E_welectron_det_100.drop(np.where(E_welectron_det_100==0)[0])
E_wproton_det_100 = E_wproton_det_100.drop(np.where(E_wproton_det_100==0)[0])
E_wneutron_det_100 = E_wneutron_det_100.drop(np.where(E_wneutron_det_100==0)[0])
E_wphoton_det_100 = E_wphoton_det_100.drop(np.where(E_wphoton_det_100==0)[0])
E_wtotal_det_100 = E_wtotal_det_100.drop(np.where(E_wtotal_det_100==0)[0])

E_welectron_det_10g = E_welectron_det_10g.drop(np.where(E_welectron_det_10g==0)[0])
E_wproton_det_10g = E_wproton_det_10g.drop(np.where(E_wproton_det_10g==0)[0])
E_wneutron_det_10g = E_wneutron_det_10g.drop(np.where(E_wneutron_det_10g==0)[0])
E_wphoton_det_10g = E_wphoton_det_10g.drop(np.where(E_wphoton_det_10g==0)[0])
E_wtotal_det_10g = E_wtotal_det_10g.drop(np.where(E_wtotal_det_10g==0)[0])

# Calibrate the energy
E_welectron_cal_100 = np.mean(E_true_100[:len(E_welectron_det_100)]/np.asarray(E_welectron_det_100))*E_welectron_det_100 
E_wproton_cal_100 = np.mean(E_true_100[:len(E_wproton_det_100)]/np.asarray(E_wproton_det_100))*E_wproton_det_100
E_wneutron_cal_100 = np.mean(E_true_100[:len(E_wneutron_det_100)]/np.asarray(E_wneutron_det_100))*E_wneutron_det_100
E_wphoton_cal_100 = np.mean(E_true_100[:len(E_wphoton_det_100)]/np.asarray(E_wphoton_det_100))*E_wphoton_det_100
E_wtotal_cal_100 = np.mean(np.asarray(4*list(E_true_100))[:len(E_wtotal_det_100)]/np.asarray(E_wtotal_det_100))*E_wtotal_det_100

E_welectron_cal_10g = np.mean(E_true_10g[:len(E_welectron_det_10g)]/np.asarray(E_welectron_det_10g))*E_welectron_det_10g 
E_wproton_cal_10g = np.mean(E_true_10g[:len(E_wproton_det_10g)]/np.asarray(E_wproton_det_10g))*E_wproton_det_10g
E_wneutron_cal_10g = np.mean(E_true_10g[:len(E_wneutron_det_10g)]/np.asarray(E_wneutron_det_10g))*E_wneutron_det_10g
E_wphoton_cal_10g = np.mean(E_true_10g[:len(E_wphoton_det_10g)]/np.asarray(E_wphoton_det_10g))*E_wphoton_det_10g
E_wtotal_cal_10g = np.mean(np.asarray(4*list(E_true_10g))[:len(E_gtotal_det_10g)]/np.asarray(E_wtotal_det_10g))*E_wtotal_det_10g

In [ ]:
# Calculating the weighted differences of the calibrated and true energy
electron_wdiff_100 = (np.asarray(E_welectron_cal_100)-E_true_100[:len(E_welectron_det_100)])/E_true_100[:len(E_welectron_det_100)]
proton_wdiff_100 = (np.asarray(E_wproton_cal_100)-E_true_100[:len(E_wproton_det_100)])/E_true_100[:len(E_wproton_det_100)]
neutron_wdiff_100 = (np.asarray(E_wneutron_cal_100)-E_true_100[:len(E_wneutron_det_100)])/E_true_100[:len(E_wneutron_det_100)]
photon_wdiff_100 = (np.asarray(E_wphoton_cal_100)-E_true_100[:len(E_wphoton_det_100)])/E_true_100[:len(E_wphoton_det_100)]
total_wdiff_100 = (np.asarray(E_wtotal_cal_100)-np.asarray(4*list(E_true_100))[:len(E_wtotal_det_100)])\
                    /np.asarray(4*list(E_true_100))[:len(E_wtotal_det_100)]

electron_wdiff_10g = (np.asarray(E_welectron_cal_10g)-E_true_10g[:len(E_welectron_det_10g)])/E_true_10g[:len(E_welectron_det_10g)]
proton_wdiff_10g = (np.asarray(E_wproton_cal_10g)-E_true_10g[:len(E_wproton_det_10g)])/E_true_10g[:len(E_wproton_det_10g)]
neutron_wdiff_10g = (np.asarray(E_wneutron_cal_10g)-E_true_10g[:len(E_wneutron_det_10g)])/E_true_10g[:len(E_wneutron_det_10g)]
photon_wdiff_10g = (np.asarray(E_wphoton_cal_10g)-E_true_10g[:len(E_wphoton_det_10g)])/E_true_10g[:len(E_wphoton_det_10g)]
total_wdiff_10g = (np.asarray(E_wtotal_cal_10g)-np.asarray(4*list(E_true_10g))[:len(E_wtotal_det_10g)])\
                    /np.asarray(4*list(E_true_10g))[:len(E_wtotal_det_10g)]

# Calculating the energy resolution
electron_wres_100 = np.std(electron_wdiff_100)
proton_wres_100 = np.std(proton_wdiff_100)
neutron_wres_100 = np.std(neutron_wdiff_100)
photon_wres_100 = np.std(photon_wdiff_100)
total_wres_100 = np.std(total_wdiff_100)

electron_wres_10g = np.std(electron_wdiff_10g)
proton_wres_10g = np.std(proton_wdiff_10g)
neutron_wres_10g = np.std(neutron_wdiff_10g)
photon_wres_10g = np.std(photon_wdiff_10g)
total_wres_10g = np.std(total_wdiff_10g)

#Plotting the Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

ax[0,0].hist(electron_wdiff_100,label='100 MeV Resolution = {:.4f}'.format(electron_wres_100))
ax[0,0].hist(electron_wdiff_10g,label='10 GeV Resolution = {:.4f}'.format(electron_wres_10g))
ax[0,0].set_title("Calibration Energy histogram of Electrons")
ax[0,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_ylabel("Frequency")
ax[0,0].legend()

ax[1,0].hist(proton_wdiff_100,label='100 MeV Resolution = {:.4f}'.format(proton_wres_100))
ax[1,0].hist(proton_wdiff_10g,label='10 GeV Resolution = {:.4f}'.format(proton_wres_10g),alpha=0.8)
ax[1,0].set_title("Calibration Energy histogram of Protons")
ax[1,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_ylabel("Frequency")
ax[1,0].set_xscale('log')
ax[1,0].legend()

ax[0,1].hist(neutron_wdiff_100,label='100 MeV Resolution = {:.4f}'.format(neutron_wres_100))
ax[0,1].hist(neutron_wdiff_10g,label='10 GeV Resolution = {:.4f}'.format(neutron_wres_10g),alpha=0.8)
ax[0,1].set_title("Calibration Energy histogram of Neutrons")
ax[0,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_ylabel("Frequency")
ax[0,1].set_xscale('log')
ax[0,1].legend()

ax[1,1].hist(photon_wdiff_100,label='100 MeV Resolution = {:.4f}'.format(photon_wres_100))
ax[1,1].hist(photon_wdiff_10g,label='10 GeV Resolution = {:.4f}'.format(photon_wres_10g))
ax[1,1].set_title("Calibration Energy histogram of Photons")
ax[1,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_ylabel("Frequency")
ax[1,1].legend()

plt.tight_layout()
plt.show()

plt.hist(total_wdiff_100,label='100 MeV Resolution = {:.4f}'.format(total_wres_100))
plt.hist(total_wdiff_10g,label='10 GeV Resolution = {:.4f}'.format(total_wres_10g),alpha=0.8)
plt.title("Calibration Energy histogram of All Particles")
plt.xlabel("($E_{cal}-E_{true})/E_{true}$")
plt.ylabel("Frequency")
plt.xscale('log')
plt.legend()
plt.show()

The energy resolution for the high energy particles remains largely unchanged as they are still able to penetrate the target whereas the energy resolution for 100 MeV is significantly higher. This is due to the fact that the tungsten absorbs most of the particles at that energy so many of them deposit no energy at all! As a result, we have the topsy-turvy scenario of the 10 GeV particles having a higher energy resolution than the 100 MeV particles. For the neutrons, however, it still doesn't hold as not even tungsten is dense enough to stop them from going through everything. Because they influence the energy resolution the most, the overall energy resolution is still better for 100 MeV particles -- but only by a factor of ~7.

## Using Beryllium

In [ ]:
# Importing the data files -- All energies are in MeV

# Get rid of column 0 which contains the true energy
electrons_100mev_Betarget = pd.read_csv( "Electrons_100mev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

protons_100mev_Betarget = pd.read_csv( "Protons_100mev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

neutrons_100mev_Betarget = pd.read_csv( "Neutrons_100mev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

photons_100mev_Betarget = pd.read_csv( "Photons_100mev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

electrons_10gev_Betarget = pd.read_csv( "Electrons_10gev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

protons_10gev_Betarget = pd.read_csv( "Protons_10gev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

neutrons_10gev_Betarget = pd.read_csv( "Neutrons_10gev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

photons_10gev_Betarget = pd.read_csv( "Photons_10gev_Betarget.csv", comment="#",usecols=[1,2,3,4,5,6],\
names=[ "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT" ] )

In [ ]:
# Append the data alongside the non-target data.

beryllium_electrons_100mev = pd.concat([electrons_100mev,electrons_100mev_Betarget],axis=1)
beryllium_protons_100mev = pd.concat([protons_100mev,protons_100mev_Betarget],axis=1)
beryllium_neutrons_100mev = pd.concat([neutrons_100mev,neutrons_100mev_Betarget],axis=1)
beryllium_photons_100mev = pd.concat([photons_100mev,photons_100mev_Betarget],axis=1)
beryllium_electrons_10gev = pd.concat([electrons_10gev,electrons_10gev_Betarget],axis=1)
beryllium_protons_10gev = pd.concat([protons_10gev,protons_10gev_Betarget],axis=1)
beryllium_neutrons_10gev = pd.concat([neutrons_10gev,neutrons_10gev_Betarget],axis=1)
beryllium_photons_10gev = pd.concat([photons_10gev,photons_10gev_Betarget],axis=1)

In [ ]:
# Append all the dataframes together by energy and shuffle to create our dataset

#Appending
beryllium_particles_100mev = pd.concat([beryllium_electrons_100mev,beryllium_protons_100mev,beryllium_neutrons_100mev,\
                                  beryllium_photons_100mev],ignore_index=True)
beryllium_particles_10gev = pd.concat([beryllium_electrons_10gev,beryllium_protons_10gev,beryllium_neutrons_10gev,\
                                 beryllium_photons_10gev],ignore_index=True)

#Shuffling
beryllium_particles_100mev = beryllium_particles_100mev.sample(frac=1).reset_index(drop=True)
beryllium_particles_10gev = beryllium_particles_10gev.sample(frac=1).reset_index(drop=True)

In [ ]:
# Check to see if our data frame looks good
beryllium_particles_100mev
# It looks great!

In [ ]:
# Create our testing and training datasets

Bet_y_100 = beryllium_particles_100mev[["Particle ID"]]
Bet_x_100 = beryllium_particles_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                           "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]]

Bet_y_10 = beryllium_particles_10gev[["Particle ID"]]
Bet_x_10 = beryllium_particles_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                         "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]]

Bet_x100m_train, Bet_x100m_test, Bet_y100m_train, Bet_y100m_test = train_test_split(Bet_x_100,Bet_y_100, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

Bet_x10g_train, Bet_x10g_test, Bet_y10g_train, Bet_y10g_test = train_test_split(Bet_x_10,Bet_y_10, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

In [ ]:
# Use the same model structure
model_Bet_100m = model(np.shape(Bet_x100m_train)[1])
model_Bet_10g = model(np.shape(Bet_x10g_train)[1])

model_Bet_100m.summary()
# This looks the same so all is well 

In [ ]:
# Train for 100 epochs with 128 batch size and 20% validation split
history_Bet_100m = model_Bet_100m.fit(x=Bet_x100m_train,y=Bet_y100m_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

# Train for 100 epochs with 128 batch size and 20% validation split
history_Bet_10g = model_Bet_10g.fit(x=Bet_x10g_train,y=Bet_y10g_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

In [ ]:
#Plotting the loss curves

fig,ax = plt.subplots(2,2,figsize=(12,9))
ax[0,0].plot(history_Bet_100m.history['loss'],label='Loss')
ax[0,0].plot(history_Bet_100m.history['val_loss'],label='Validation Loss')
ax[0,0].set_title("Loss Change over Time (100 MeV)")
ax[0,0].set_xlabel("Epoch")
ax[0,0].set_ylabel("Loss")
ax[0,0].legend()

ax[1,0].plot(history_Bet_100m.history['accuracy'],label='Accuracy')
ax[1,0].plot(history_Bet_100m.history['val_accuracy'],label='Validation Accuracy')
ax[1,0].set_title("Accuracy Change over Time (100 MeV)")
ax[1,0].set_xlabel("Epoch")
ax[1,0].set_ylabel("Accuracy")
ax[1,0].legend()

ax[0,1].plot(history_Bet_10g.history['loss'],label='Loss')
ax[0,1].plot(history_Bet_10g.history['val_loss'],label='Validation Loss')
ax[0,1].set_title("Loss Change over Time (10 GeV)")
ax[0,1].set_xlabel("Epoch")
ax[0,1].set_ylabel("Loss")
ax[0,1].legend()

ax[1,1].plot(history_Bet_10g.history['accuracy'],label='Accuracy')
ax[1,1].plot(history_Bet_10g.history['val_accuracy'],label='Validation Accuracy')
ax[1,1].set_title("Accuracy Change over Time (10 GeV)")
ax[1,1].set_xlabel("Epoch")
ax[1,1].set_ylabel("Accuracy")
ax[1,1].legend()

plt.tight_layout()
plt.show()

From these loss curves, like graphite, it is possible to see that the beryllium target is most beneficial for distinguishing low energy particles. Despite not having an accuracy as high as graphite, it is still superior to the original set-up and the lines are pretty stable and show no signs of overtraining meaning the batch size and epoch number is justified.  

In [ ]:
# Testing the model on the testing datasets
prediction_Bet_100m = np.argmax(model_Bet_100m.predict(Bet_x100m_test),axis=1)
prediction_Bet_10g = np.argmax(model_Bet_10g.predict(Bet_x10g_test),axis=1)

In [ ]:
#Plot confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig,ax = plt.subplots(1,2,figsize=(12,5))
#Define the confusion matrix and normalise it so that each row/column sums to 1
Bet_cmat100m = confusion_matrix(Bet_y100m_test,prediction_Bet_100m,normalize='true')
Bet_cmat10g = confusion_matrix(Bet_y10g_test,prediction_Bet_10g,normalize='true')
#dispay the matrix and redefine the display labels to show the particle type
Bet_cmatplot100m = ConfusionMatrixDisplay(confusion_matrix=Bet_cmat100m,display_labels=['electron','proton','neutron','photon'])
Bet_cmatplot10g = ConfusionMatrixDisplay(confusion_matrix=Bet_cmat10g,display_labels=['electron','proton','neutron','photon'])

Bet_cmatplot100m.plot(ax=ax[0])
ax[0].set_title("100 MeV")
Bet_cmatplot10g.plot(ax=ax[1])
ax[1].set_title("10 GeV")
plt.show()

These confusion matrices are comparable to those of graphite as the distinction between hadrons and elementary particles in enhanced compared to the original matrix and the distinction amongst the hadrons themselves is also increased by a similar amount. The only difference between the graphite matrices and these ones large enough worth mentioning is the fact that the photon is detected worse here than with graphite. It is unclear what the reason is but the confusion between photons and electrons is much higher this time. It could be the electronic/atomic structure of beryllium that causes the photons to pair produce positron-electron pairs that then enter the calorimeter disguised as electrons. Because they have lost a similar amount of energy to the electrons, it is possible that they deposit very similar amounts of energy in the homogeneous calorimeter. 

## Energy Resolution

In [ ]:
# Calculating the energy resolution for each particle separately and then all of them together

# Getting the data required 
E_true_100 = electrons_100mev['True Energy'] # Same for every particle
E_true_10g = electrons_10gev['True Energy']
# Total detected energy
E_Beelectron_det_100 = beryllium_electrons_100mev[["HomogeneousBeT","Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]].sum(axis=1)
E_Beproton_det_100 = beryllium_protons_100mev[["HomogeneousBeT","Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]].sum(axis=1)
E_Beneutron_det_100 = beryllium_neutrons_100mev[["HomogeneousBeT","Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]].sum(axis=1)
E_Bephoton_det_100 = beryllium_photons_100mev[["HomogeneousBeT","Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]].sum(axis=1)
E_Betotal_det_100 = pd.concat([E_Beelectron_det_100,E_Beproton_det_100,E_Beneutron_det_100,E_Bephoton_det_100],\
                           ignore_index=True)

E_Beelectron_det_10g = beryllium_electrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_Beproton_det_10g = beryllium_protons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_Beneutron_det_10g = beryllium_neutrons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_Bephoton_det_10g = beryllium_photons_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5"]].sum(axis=1)
E_Betotal_det_10g = pd.concat([E_Beelectron_det_10g,E_Beproton_det_10g,E_Beneutron_det_10g,E_Bephoton_det_10g],\
                           ignore_index=True)

# Remove zero values to avoid divide by 0 errors
E_Beelectron_det_100 = E_Beelectron_det_100.drop(np.where(E_Beelectron_det_100==0)[0])
E_Beproton_det_100 = E_Beproton_det_100.drop(np.where(E_Beproton_det_100==0)[0])
E_Beneutron_det_100 = E_Beneutron_det_100.drop(np.where(E_Beneutron_det_100==0)[0])
E_Bephoton_det_100 = E_Bephoton_det_100.drop(np.where(E_Bephoton_det_100==0)[0])
E_Betotal_det_100 = E_Betotal_det_100.drop(np.where(E_Betotal_det_100==0)[0])

E_Beelectron_det_10g = E_Beelectron_det_10g.drop(np.where(E_Beelectron_det_10g==0)[0])
E_Beproton_det_10g = E_Beproton_det_10g.drop(np.where(E_Beproton_det_10g==0)[0])
E_Beneutron_det_10g = E_Beneutron_det_10g.drop(np.where(E_Beneutron_det_10g==0)[0])
E_Bephoton_det_10g = E_Bephoton_det_10g.drop(np.where(E_Bephoton_det_10g==0)[0])
E_Betotal_det_10g = E_Betotal_det_10g.drop(np.where(E_Betotal_det_10g==0)[0])

# Calibrate the energy
E_Beelectron_cal_100 = np.mean(E_true_100[:len(E_Beelectron_det_100)]/np.asarray(E_Beelectron_det_100))*E_Beelectron_det_100 
E_Beproton_cal_100 = np.mean(E_true_100[:len(E_Beproton_det_100)]/np.asarray(E_Beproton_det_100))*E_Beproton_det_100
E_Beneutron_cal_100 = np.mean(E_true_100[:len(E_Beneutron_det_100)]/np.asarray(E_Beneutron_det_100))*E_Beneutron_det_100
E_Bephoton_cal_100 = np.mean(E_true_100[:len(E_Bephoton_det_100)]/np.asarray(E_Bephoton_det_100))*E_Bephoton_det_100
E_Betotal_cal_100 = np.mean(np.asarray(4*list(E_true_100))[:len(E_Betotal_det_100)]/np.asarray(E_Betotal_det_100))*E_Betotal_det_100

E_Beelectron_cal_10g = np.mean(E_true_10g[:len(E_Beelectron_det_10g)]/np.asarray(E_Beelectron_det_10g))*E_Beelectron_det_10g 
E_Beproton_cal_10g = np.mean(E_true_10g[:len(E_Beproton_det_10g)]/np.asarray(E_Beproton_det_10g))*E_Beproton_det_10g
E_Beneutron_cal_10g = np.mean(E_true_10g[:len(E_Beneutron_det_10g)]/np.asarray(E_Beneutron_det_10g))*E_Beneutron_det_10g
E_Bephoton_cal_10g = np.mean(E_true_10g[:len(E_Bephoton_det_10g)]/np.asarray(E_Bephoton_det_10g))*E_Bephoton_det_10g
E_Betotal_cal_10g = np.mean(np.asarray(4*list(E_true_10g))[:len(E_gtotal_det_10g)]/np.asarray(E_Betotal_det_10g))*E_Betotal_det_10g

In [ ]:
# Calculating the weighted differences of the calibrated and true energy
electron_Bediff_100 = (np.asarray(E_Beelectron_cal_100)-E_true_100[:len(E_Beelectron_det_100)])/E_true_100[:len(E_Beelectron_det_100)]
proton_Bediff_100 = (np.asarray(E_Beproton_cal_100)-E_true_100[:len(E_Beproton_det_100)])/E_true_100[:len(E_Beproton_det_100)]
neutron_Bediff_100 = (np.asarray(E_Beneutron_cal_100)-E_true_100[:len(E_Beneutron_det_100)])/E_true_100[:len(E_Beneutron_det_100)]
photon_Bediff_100 = (np.asarray(E_Bephoton_cal_100)-E_true_100[:len(E_Bephoton_det_100)])/E_true_100[:len(E_Bephoton_det_100)]
total_Bediff_100 = (np.asarray(E_Betotal_cal_100)-np.asarray(4*list(E_true_100))[:len(E_Betotal_det_100)])\
                    /np.asarray(4*list(E_true_100))[:len(E_Betotal_det_100)]

electron_Bediff_10g = (np.asarray(E_Beelectron_cal_10g)-E_true_10g[:len(E_Beelectron_det_10g)])/E_true_10g[:len(E_Beelectron_det_10g)]
proton_Bediff_10g = (np.asarray(E_Beproton_cal_10g)-E_true_10g[:len(E_Beproton_det_10g)])/E_true_10g[:len(E_Beproton_det_10g)]
neutron_Bediff_10g = (np.asarray(E_Beneutron_cal_10g)-E_true_10g[:len(E_Beneutron_det_10g)])/E_true_10g[:len(E_Beneutron_det_10g)]
photon_Bediff_10g = (np.asarray(E_Bephoton_cal_10g)-E_true_10g[:len(E_Bephoton_det_10g)])/E_true_10g[:len(E_Bephoton_det_10g)]
total_Bediff_10g = (np.asarray(E_Betotal_cal_10g)-np.asarray(4*list(E_true_10g))[:len(E_Betotal_det_10g)])\
                    /np.asarray(4*list(E_true_10g))[:len(E_Betotal_det_10g)]

# Calculating the energy resolution
electron_Beres_100 = np.std(electron_Bediff_100)
proton_Beres_100 = np.std(proton_Bediff_100)
neutron_Beres_100 = np.std(neutron_Bediff_100)
photon_Beres_100 = np.std(photon_Bediff_100)
total_Beres_100 = np.std(total_Bediff_100)

electron_Beres_10g = np.std(electron_Bediff_10g)
proton_Beres_10g = np.std(proton_Bediff_10g)
neutron_Beres_10g = np.std(neutron_Bediff_10g)
photon_Beres_10g = np.std(photon_Bediff_10g)
total_Beres_10g = np.std(total_Bediff_10g)

#Plotting the Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

ax[0,0].hist(electron_Bediff_100,label='100 MeV Resolution = {:.4f}'.format(electron_Beres_100))
ax[0,0].hist(electron_Bediff_10g,label='10 GeV Resolution = {:.4f}'.format(electron_Beres_10g),alpha=0.8)
ax[0,0].set_title("Calibration Energy histogram of Electrons")
ax[0,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_ylabel("Frequency")
ax[0,0].legend()

ax[1,0].hist(proton_Bediff_100,label='100 MeV Resolution = {:.4f}'.format(proton_Beres_100))
ax[1,0].hist(proton_Bediff_10g,label='10 GeV Resolution = {:.4f}'.format(proton_Beres_10g))
ax[1,0].set_title("Calibration Energy histogram of Protons")
ax[1,0].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_ylabel("Frequency")
ax[1,0].legend()

ax[0,1].hist(neutron_Bediff_100,label='100 MeV Resolution = {:.4f}'.format(neutron_Beres_100))
ax[0,1].hist(neutron_Bediff_10g,label='10 GeV Resolution = {:.4f}'.format(neutron_Beres_10g),alpha=0.8)
ax[0,1].set_title("Calibration Energy histogram of Neutrons")
ax[0,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_ylabel("Frequency")
ax[0,1].set_xscale('log')
ax[0,1].legend()

ax[1,1].hist(photon_Bediff_100,label='100 MeV Resolution = {:.4f}'.format(photon_Beres_100))
ax[1,1].hist(photon_Bediff_10g,label='10 GeV Resolution = {:.4f}'.format(photon_Beres_10g))
ax[1,1].set_title("Calibration Energy histogram of Photons")
ax[1,1].set_xlabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_ylabel("Frequency")
ax[1,1].legend()

plt.tight_layout()
plt.show()

plt.hist(total_Bediff_100,label='100 MeV Resolution = {:.4f}'.format(total_Beres_100))
plt.hist(total_Bediff_10g,label='10 GeV Resolution = {:.4f}'.format(total_Beres_10g),alpha=0.8)
plt.title("Calibration Energy histogram of all Particles")
plt.xlabel("($E_{cal}-E_{true})/E_{true}$")
plt.ylabel("Frequency")
plt.xscale('log')
plt.legend()
plt.show()

For 100 MeV, the overall energy resolution is much better than tungsten although much worse than graphite by a factor 2. Again, the high neutron resolution dominates the total resolution but the individual resolutions show that photons, electrons and protons do have a higher resolution than graphite. This more or less disproves the theory mentioned above (where photon pair production causes the confusion) as if it were the case, then a worse electron and photon resolution would be expected here. This is not the case. Thus, it appears that graphite is not (marginally) better than beryllium due to the amount of particles that elastically scatter but by the number of particles that inelastically scatter. Thus, using a fixed target experiment on a linear detector, it appears that the quality of particle classification is inversely proportional to the energy resolution.  

## Putting all data together
 In this section, the possibility of improving particle classification by using all of the target data simultaneously is investigated.

In [ ]:
# Concatenating all the target data
all_electrons_100mev = pd.concat([electrons_100mev,electrons_100mev_Ctarget,electrons_100mev_Wtarget,\
                                 electrons_100mev_Betarget],axis=1)
all_protons_100mev = pd.concat([protons_100mev,protons_100mev_Ctarget,protons_100mev_Wtarget,\
                               protons_100mev_Betarget],axis=1)
all_neutrons_100mev = pd.concat([neutrons_100mev,neutrons_100mev_Ctarget,neutrons_100mev_Wtarget,\
                                neutrons_100mev_Betarget],axis=1)
all_photons_100mev = pd.concat([photons_100mev,photons_100mev_Ctarget,photons_100mev_Wtarget,\
                               photons_100mev_Betarget],axis=1)
all_electrons_10gev = pd.concat([electrons_10gev,electrons_10gev_Ctarget,electrons_10gev_Wtarget,\
                                electrons_10gev_Betarget],axis=1)
all_protons_10gev = pd.concat([protons_10gev,protons_10gev_Ctarget,protons_10gev_Wtarget,\
                              protons_10gev_Betarget],axis=1)
all_neutrons_10gev = pd.concat([neutrons_10gev,neutrons_10gev_Ctarget,neutrons_10gev_Wtarget,\
                               neutrons_10gev_Betarget],axis=1)
all_photons_10gev = pd.concat([photons_10gev,photons_10gev_Ctarget,photons_10gev_Wtarget,\
                              photons_10gev_Betarget],axis=1)

In [ ]:
#Appending them all together
all_particles_100mev = pd.concat([all_electrons_100mev,all_protons_100mev,all_neutrons_100mev,\
                                  all_photons_100mev],ignore_index=True)
all_particles_10gev = pd.concat([all_electrons_10gev,all_protons_10gev,all_neutrons_10gev,\
                                 all_photons_10gev],ignore_index=True)

#Randomly shuffling
all_particles_100mev = all_particles_100mev.sample(frac=1).reset_index(drop=True)
all_particles_10gev = all_particles_10gev.sample(frac=1).reset_index(drop=True)

In [ ]:
# Check to see if our data frame looks good
all_particles_100mev
# It looks great!

In [ ]:
#Creating the testing and training data sets

all_y_100 = all_particles_100mev[["Particle ID"]]
all_x_100 = all_particles_100mev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                           "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT",\
                           "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT",\
                           "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]]

all_y_10 = all_particles_10gev[["Particle ID"]]
all_x_10 = all_particles_10gev[["Homogeneous","Layer 1","Layer 2", "Layer 3","Layer 4","Layer 5",\
                           "HomogeneousCT", "Layer 1CT","Layer 2CT", "Layer 3CT","Layer 4CT","Layer 5CT",\
                           "HomogeneousWT", "Layer 1WT","Layer 2WT", "Layer 3WT","Layer 4WT","Layer 5WT",\
                           "HomogeneousBeT", "Layer 1BeT","Layer 2BeT", "Layer 3BeT","Layer 4BeT","Layer 5BeT"]]

all_x100m_train, all_x100m_test, all_y100m_train, all_y100m_test = train_test_split(all_x_100,all_y_100, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

all_x10g_train, all_x10g_test, all_y10g_train, all_y10g_test = train_test_split(all_x_10,all_y_10, test_size=0.3,
                                                        random_state=1) # 70% training and 30% test

In [ ]:
# Use the same model architecture as for all the others
model_all_100m = model(np.shape(all_x100m_train)[1])
model_all_10g = model(np.shape(all_x10g_train)[1])

model_all_100m.summary()
# This has the expected increase in parameters so all is well 

In [ ]:
# Use 100 epochs with 20% validation data and a batch size of 128
history_all_100m = model_all_100m.fit(x=all_x100m_train,y=all_y100m_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

# Use 100 epochs with 20% validation data and a batch size of 128
history_all_10g = model_all_10g.fit(x=all_x10g_train,y=all_y10g_train,batch_size=128,\
                                              epochs=100,validation_split=0.2,shuffle=True)

In [ ]:
#Plotting the loss curves

fig,ax = plt.subplots(2,2,figsize=(12,9))
ax[0,0].plot(history_all_100m.history['loss'],label='Loss')
ax[0,0].plot(history_all_100m.history['val_loss'],label='Validation Loss')
ax[0,0].set_title("Loss Change over Time (100 MeV)")
ax[0,0].set_xlabel("Epoch")
ax[0,0].set_ylabel("Loss")
ax[0,0].legend()

ax[1,0].plot(history_all_100m.history['accuracy'],label='Accuracy')
ax[1,0].plot(history_all_100m.history['val_accuracy'],label='Validation Accuracy')
ax[1,0].set_title("Accuracy Change over Time (100 MeV)")
ax[1,0].set_xlabel("Epoch")
ax[1,0].set_ylabel("Accuracy")
ax[1,0].legend()

ax[0,1].plot(history_all_10g.history['loss'],label='Loss')
ax[0,1].plot(history_all_10g.history['val_loss'],label='Validation Loss')
ax[0,1].set_title("Loss Change over Time (10 GeV)")
ax[0,1].set_xlabel("Epoch")
ax[0,1].set_ylabel("Loss")
ax[0,1].legend()

ax[1,1].plot(history_all_10g.history['accuracy'],label='Accuracy')
ax[1,1].plot(history_all_10g.history['val_accuracy'],label='Validation Accuracy')
ax[1,1].set_title("Accuracy Change over Time (10 GeV)")
ax[1,1].set_xlabel("Epoch")
ax[1,1].set_ylabel("Accuracy")
ax[1,1].legend()

plt.tight_layout()
plt.show()

These loss curves are vastly superior to all the others; they show no signs of overtraining, and both have an accuracy of around 90%. Like the others (except for tungsten), the accuracy is higher for low energies than for high energies which is probably due to the fact that the combination of beryllium and graphite were instrumental in differentiating them.

In [ ]:
# Predicting the testing data sets
prediction_all_100m = np.argmax(model_all_100m.predict(all_x100m_test),axis=1)
prediction_all_10g = np.argmax(model_all_10g.predict(all_x10g_test),axis=1)

In [ ]:
#Plot confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig,ax = plt.subplots(1,2,figsize=(12,5))
#Define the confusion matrix and normalise it so that each row/column sums to 1
all_cmat100m = confusion_matrix(all_y100m_test,prediction_all_100m,normalize='true')
all_cmat10g = confusion_matrix(all_y10g_test,prediction_all_10g,normalize='true')
#dispay the matrix and redefine the display labels to show the particle type
all_cmatplot100m = ConfusionMatrixDisplay(confusion_matrix=all_cmat100m,display_labels=['electron','proton','neutron','photon'])
all_cmatplot10g = ConfusionMatrixDisplay(confusion_matrix=all_cmat10g,display_labels=['electron','proton','neutron','photon'])

all_cmatplot100m.plot(ax=ax[0])
ax[0].set_title("100 MeV")
all_cmatplot10g.plot(ax=ax[1])
ax[1].set_title("10 GeV")
plt.show()

These confusion matrices are much better than all the others for both energies. At 100 MeV, the distinction of photons is at 100% and the neutrons are almost as high. With an 80% accuracy of both electrons and photons, it can be seen that the combination of the 4 data types were better than when each were separate. This is to be expected as there is more data but the large increase is also to do with the interactions that occured on the targets. Looking at the energy calibration, it appears more energy was deposited in the target by neutrons when it was beryllium (resolution of 14 vs 18 for beryllium) and more energy was deposited by photons, electrons and protons when it was graphite. Thus, by combining the data, all particles could be looked at and the high energy confusion matrix was somewhat helped by the tungsten which only let through the highest energy particles.

The photons and electrons are still getting confused but this is a classic issue in particle physics as they do interact very similarly. As a further extension, adding a magnetic field and a tracker would be even better as then curvature could be used on top of energy calculations.